In [ ]:
import numpy as np
import math
import holoviews as hv
hv.extension('bokeh')
from bokeh.io import show, curdoc, output_notebook
from bokeh.layouts import layout, widgetbox
from bokeh.models import Slider, Button, TextInput, Select
output_notebook()

### Funções Geométricas

In [ ]:
def create_points(n=3, mu=0, std=1):
    x = np.random.normal(mu, std, n)
    y = np.random.normal(mu, std, n)
    points = []
    for i in range(len(x)):
        points.append((x[i], y[i]))
    points = list(set(points))
    return points

def distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def slope(p1, p2):
    y = (p1[1] - p2[1])
    x = (p1[0] - p2[0])
    return math.atan2(y, x) 
    
def cross_product(p1, p2, p3):
    return ((p2[0] - p1[0]) * (p3[1] - p1[1])) - ((p2[1] - p1[1]) * (p3[0] - p1[0]))

def relative_orientation(p1, p2, p3):
    x = (p2[0]-p1[0], p2[1]-p1[1])
    y = (p3[0]-p1[0], p3[1]-p1[1])
    return (x[0]*y[1]) - (y[0]*x[1])

### Funções de Animação

In [ ]:
renderer = hv.renderer('bokeh').instance(mode='server')
scatter = None
steps = None
end = None
dmap = None
stream = None
N = None
MU = None
STD = None
alg = None

def ch(it):
    global steps
    global scatter
    return scatter * steps.get(it)

def modify_plot(doc):
    global dmap
    hvplot = renderer.get_plot(dmap, doc)

    def animate_update():
        it = slider.value + 1
        if it > end:
            it = start
        slider.value = it

    def slider_update(attrname, old, new):
        global stream
        stream.event(it=new)
    
    global end
    start = 0
    slider = Slider(start=start, end=end, value=start, step=1, title="Iteration")
    slider.on_change('value', slider_update)
    
    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update, 500)
        else:
            button.label = '► Play'
            doc.remove_periodic_callback(callback_id)
    button = Button(label='► Play', width=60)
    button.on_click(animate)
    
    plot = layout([
    [hvplot.state],
    [slider, button]], sizing_mode='fixed')
    
    doc.add_root(plot)
    return doc

In [ ]:
def ch_plot(points, alg):
    global scatter
    global steps
    global end
    global dmap
    global stream

    scatter = hv.Scatter(points)
    hull, steps = alg(points)
    end = len(steps) - 1
    stream = hv.streams.Stream.define('Iteration', it=0)()
    dmap = hv.DynamicMap(ch, streams=[stream])

    show(modify_plot)

In [ ]:
def modify_param(doc):
    global N
    global MU
    global STD
    N = None
    MU = None
    STD = None
    
    n_param = TextInput(title="N", value='')
    def update_n(attrname, old, new):
        global N
        N = int(n_param.value)

    n_param.on_change('value', update_n)
    
    mu_param = TextInput(title="MU", value='') 
    def update_mu(attrname, old, new):
        global MU
        MU = float(mu_param.value)

    mu_param.on_change('value', update_mu)
    
    std_param = TextInput(title="STD", value='')
    def update_std(attrname, old, new):
        global STD
        STD = float(std_param.value)

    std_param.on_change('value', update_std)

    select = Select(title="Algoritimo",  options=['Graham', 'Jarvis', 'Incremental'])
    def update_select(attrname, old, new):
        global alg
        if select.value == 'Graham':
            alg = graham_scan
        elif select.value == 'Jarvis':
            alg = gift_wrapping
        elif select.value == 'Incremental':
            alg = incremental_v2
            
    select.on_change('value', update_select)
    
    plot_button = Button(label="Plot it!", button_type="success")
    def update_plot():
        global N
        global MU
        global STD
        global alg
        if N == None or MU == None or STD == None:
            points = create_points() 
            ch_plot(points, alg)
        else:
            points = create_points(N, MU, STD)
            ch_plot(points, alg)
        
    plot_button.on_click(update_plot)
    
    plot = layout([
    [n_param, mu_param, std_param], 
    [select, plot_button]], sizing_mode='fixed')
    
    doc.add_root(plot)
    return doc

### Varredura de Graham

In [ ]:
def graham_scan(points_):
    points = points_.copy()
    anchor = min(points, key=lambda p: (p[1], p[0]))
    points.pop(points.index(anchor))
    points.sort(key=lambda p: (slope(p, anchor), distance(anchor, p)))
    
    it = 0
    steps = {}
    hull = [anchor]
    steps[it] = hv.Path(hull) * hv.Path(hull)
    it += 1
    
    for p in points:
        steps[it] = hv.Path(hull) * hv.Path([hull[-1], p])
        it += 1
        hull.append(p)
        while len(hull) > 2 and cross_product(hull[-3], hull[-2], hull[-1]) < 0:
            hull.pop(-2)
            steps[it] = hv.Path(hull) * hv.Path([hull[-1], p])
            it += 1
            
    steps[it] = hv.Path(hull) * hv.Path([hull[-1], anchor])
    it += 1
    hull.append(hull[0])
    steps[it] = hv.Path(hull) * hv.Path(hull)
    
    return hull, steps

### Embrulho para presente

In [ ]:
def gift_wrapping(points):
    anchor = min(points, key=lambda p: p[0])
    start = points.index(anchor)
    p = start
    hull = []
    it = 0
    steps = {}
    
    while 1:
        hull.append(points[p])
        steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[p]])
        it += 1
        q = (p + 1) % len(points)
        for i in range(len(points)):
            steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[i]])
            it += 1
            if cross_product(points[p], points[i], points[q]) < 0:
                q = i
        p = q
        if p == start:
            break
        steps[it] = hv.Path(hull) * hv.Path([hull[-1], points[p]])
        it += 1
    
    hull.append(hull[0])
    steps[it] = hv.Path(hull) * hv.Path(hull)
    
    return hull, steps  

### Incremental (Plot not finished)

In [ ]:
def incremental_v2(points):
    points.sort(key=lambda p: p[0])
    up = {}
    low = {}
    up[0], up[1] = 1, 0
    low[0], low[1] = 1, 0
    hull = []
    
    it = 0
    steps = {}
    hull.append(points[0])
    steps[it] = hv.Path(hull) * hv.Path(hull)
    it += 1
    hull.append(points[1])
    steps[it] = hv.Path(hull) * hv.Path(hull)
    it += 1
    anchor = min(hull, key=lambda p: (p[1], p[0]))
    hull_polar = sorted(hull, key=lambda p: slope(p, anchor))
    
    for i in range(2, len(points)):
        hull.append(points[i])
        anchor = min(hull, key=lambda p: (p[1], p[0]))
        hull_polar = sorted(hull, key=lambda p: slope(p, anchor))
        
        if points[i][1] > points[i-1][1]:
            up[i] = i-1
            low[i] = low[i-1]
        else:
            up[i] = up[i-1]
            low[i] = i-1
        
        up[low[i]] = i
        low[up[i]] = i
        up_n = up[i]
        up_nn = up[up[i]]
        
        while relative_orientation(points[i], points[up_n], points[up_nn]) > 0:
            up[i] = up_nn
            low[up_nn] = i
            hull = [p for p in hull if p != points[up_n]]
            anchor = min(hull, key=lambda p: (p[1], p[0]))
            hull_polar = sorted(hull, key=lambda p: slope(p, anchor))
            
            up_n = up_nn
            up_nn = up[up_n]
            
        low_p = low[i]
        low_pp = low[low[i]]
            
        while relative_orientation(points[i], points[low_pp], points[low_p]) > 0:
            low[i] = low_pp
            up[low_pp] = i
            hull = [p for p in hull if p != points[low_p]]
            anchor = min(hull, key=lambda p: (p[1], p[0]))
            hull_polar = sorted(hull, key=lambda p: slope(p, anchor))
            
            low_p = low_pp
            low_pp = low[low_p]
            

    steps[it] = hv.Points(hull) * hv.Path(hull)
    
    return hull, steps

### Plotando

In [ ]:
show(modify_param)